In [12]:
import pandas as pd
import numpy as np
import math

In [327]:
df = pd.read_csv("user_study_country-artha.csv")
df

,id,mod,text,sentiment,is_make_sense,comment
0,0,0,*** SPOILERS****** SPOILERS*** There's not muc...,Negative,Yes,NaN
1,1,1,*** SPOILERS****** SPOILERS*** There's not muc...,Negative,No,"Inconsistent reference to Kerigan and Cravat, ..."
2,2,2,*** SPOILERS****** SPOILERS*** There's not muc...,Negative,No,Inconsistent reference (LascƒÉr and some lefto...
3,3,0,That distinction has to go to THE DUNGEON OF H...,Negative,No,"Fallon seems to be female Irish name, so ""Fall..."
4,4,1,That distinction has to go to THE DUNGEON OF H...,Negative,Yes,NaN
...,...,...,...,...,...,...
145,145,1,"There is no "" fun "" poking fun at the desperat...",Negative,Yes,NaN
146,146,2,"There is no "" fun "" poking fun at the desperat...",Negative,Yes,NaN
147,147,0,*** SPOILERS****** SPOILERS*** There's not muc...,Negative,Yes,NaN
148,148,1,*** SPOILERS****** SPOILERS*** There's not muc...,Negative,No,"Inconsistent reference (""Cravat"",""Christop"")"


#### Text Preprocessing

In [13]:
from utils import preprocessText

In [14]:
text = "And one only, in my opinion.<br /><br />That reason is Margaret Leighton. She is a wonderful actress, on-stage as well as on the screen. We have few chances to see her, though. I think that\'s especially true in the United States.<br /><br />Here she plays a sympathetic role. Not only that but she is also very pretty and meant to be something of a bombshell.<br /><br />Walter Pigeon does not hold up the tradition of Drummond performers. He is always reliable but he's not much fun. He's not a rascal or a knave. Consequently, this seemed to me a talky endeavor with little action or suspense. But check it out for Leighton."
print(text)

And one only, in my opinion.<br /><br />That reason is Margaret Leighton. She is a wonderful actress, on-stage as well as on the screen. We have few chances to see her, though. I think that's especially true in the United States.<br /><br />Here she plays a sympathetic role. Not only that but she is also very pretty and meant to be something of a bombshell.<br /><br />Walter Pigeon does not hold up the tradition of Drummond performers. He is always reliable but he's not much fun. He's not a rascal or a knave. Consequently, this seemed to me a talky endeavor with little action or suspense. But check it out for Leighton.


In [15]:
preprocessText(text)

"And one only, in my opinion. That reason is Margaret Leighton. She is a wonderful actress, onstage as well as on the screen. We have few chances to see her, though. I think that's especially true in the United States. Here she plays a sympathetic role. Not only that but she is also very pretty and meant to be something of a bombshell. Walter Pigeon does not hold up the tradition of Drummond performers. He is always reliable but he's not much fun. He's not a rascal or a knave. Consequently, this seemed to me a talky endeavor with little action or suspense. But check it out for Leighton."

# Mutant Generation

In [16]:
import spacy
import pandas as pd

from utils import preprocessText
# from MutantGeneration import MutantGeneration

In [291]:
import re
import pandas as pd

from utils import NAME, PRONOUN, GAW, SALUTATION

from Entity import Entity
from Phrase import Phrase
from Coreference import Coreference
from utils import nlp
from utils import tag, preprocessText
from utils import getPronounPlaceholders
from utils import masculineToFemininePronoun
from utils import feminineToMasculinePronoun
from utils import masculineToFeminineSalutation
from utils import feminineToMasculineSalutation
from utils import getMaleNamesFromGenderComputer, getFemaleNamesFromGenderComputer
from utils import getMasculineGenderAssociatedWord, getFeminineGenderAssociatedWord 

class MutantGeneration:
    original = ""
    coreferences = []
    person_entities = []
    chunks = []
    template = ""
    examples = []
    mutants = []
    templates = []
    genders = []
    valid_coreferences = []
    
    def __init__(self, text):
        
        self.original = str(text)
        self.docs = nlp(text)
        
        self.person_entities = self.getPersonEntities()

        # self.coreferences = []
        self.person_coreferences = []
        self.person_coreferences = self.getPersonCoreferences()
        if len(self.person_coreferences) == 1 :
            coref = self.person_coreferences[0]
            if self.isValid(coref) :
                template = self.generateTemplate(coref)
                self.templates, self.mutants, self.genders = self.generateMutant(coref, template)
            

    def getPersonCoreferences(self) :
    
        coreferences = []
        for r in self.docs._.coref_clusters :
            coref = Coreference(r.main, r.mentions)
            if self.isPersonCoref(coref) : # only take valid coreference
                coreferences.append(coref)
        
        return coreferences

    def isValid(self, coref):
        placeholders = []
#         print("SELECTED COREF: " + str(coref.getReferences()))
        gender = ""
        for phrase in coref.getReferences() :
            if phrase.isGenderPronoun():
                if gender == "" :
                    gender = phrase.getGender()
                elif gender != phrase.getGender() : ## there is 2 gender pronoun detected
                    return False
                coref.setGender(phrase.getGender())
                id = phrase.getPhrase()
                placeholders.append(tag(PRONOUN + "-" + id))
            elif self.isPersonName(phrase.getPhrase()):
                placeholders.append(tag(NAME))
            elif phrase.isContainGenderAssociatedWord() :
                gaw = phrase.getGenderAssociatedWord()
                placeholder = phrase.getPhrase().replace(gaw, tag(GAW))
                placeholders.append(placeholder)
            else :
                return False

        if coref.getGender() == "" : # no pronoun found
            return False
        
        coref.setPlaceholders(placeholders)

        ## replace <name><name> into <name>
        ## how if all pronoun -> pass
        return True
            
    def isPersonCoref(self, coref):
        placeholders = []
#         print("COREF: " + str(coref.getReferences()))
        gender = ""
        for phrase in coref.getReferences() :
            if phrase.isGenderPronoun():
                return True
#                 if gender == "" :
#                     gender = phrase.getGender()
#                 elif gender != phrase.getGender() : ## there is 2 gender pronoun detected
#                     return False
#                 coref.setGender(phrase.getGender())
#                 id = phrase.getPhrase()
#                 placeholders.append(tag(PRONOUN + "-" + id))
            elif self.isPersonName(phrase.getPhrase()):
                return True
#                 placeholders.append(tag(NAME))
            elif phrase.isContainGenderAssociatedWord() :
                return True
#                 gaw = phrase.getGenderAssociatedWord()
#                 placeholder = phrase.getPhrase().replace(gaw, tag(GAW))
#                 placeholders.append(placeholder)

#         if coref.getGender() == "" : # no pronoun found
#             return False
        
#         coref.setPlaceholders(placeholders)

        ## replace <name><name> into <name>
        ## how if all pronoun -> pass
        return False

    
    def getOriginal(self):
        return self.original

    def getCoreferences(self):
        return self.coreferences
    
    def getTemplates(self):
        return self.templates
    
    def getGenders(self):
        return self.genders
    
    def getMutants(self) :
        return self.mutants
    
    def isPersonName(self, text) :
        return text in self.person_entities 

    def isContainPersonName(self, phrase) :
        tokens = phrase.getTokens()
        for token in tokens:
            if token.text in self.person_entities :
                return True
        return False

    def getPersonEntities(self) :
        entities = set()
        for ent in self.docs.ents :
            e = Entity(ent.text, ent.start_char, ent.end_char, ent.label_)
            if e.isPerson() :
                entities.add(e.getWord())
#                 print("PERSON: ", e.getWord())
#         print()
        return list(entities)
            
    def generateTemplate(self, coref) :
        chunks = self.generateChunk(coref)
        placeholders = coref.getPlaceholders()
        
        tokens = [chunks[0]]
        i = 1
        
        for placeholder in placeholders :
            tokens.append(placeholder)
            tokens.append(chunks[i])
            i += 1  

        template = " ".join(tokens).strip()
        template = re.sub(' +', ' ', template)

#         print(self.original)
#         print(chunks)
#         print(placeholders)
#         print(template)
        self.template = str(template)
        return template
    
    def getTemplate(self):
        return self.template
    
    def getMutants(self):
        return self.mutants
        
    def getMutantExamples(self):
        return self.examples

        
    def generateChunk(self, coref) :
        chunks = []
        refs = coref.getPositionReferences()
        lb = 0 # lower bound
        ub = 0 # upper bound
        for i in range(len(refs)) :
            if i == 0 :
                ub = refs[i].start
                chunk = self.original[lb:ub]
                chunks.append(chunk)
            else :
                lb = refs[i-1].end
                ub = refs[i].start
                chunk = self.original[lb:ub]
                chunks.append(chunk)
            if i == len(refs)-1 :
                lb = refs[-1].end
                chunks.append(self.original[lb:])
        return chunks
    
    
    def generateMutant(self, coref, template) :
        templates = []
        mutants = []
        genders = []
        
        self.examples = []
        placeholders = coref.getPlaceholders()
        used_placeholders = set(placeholders)
        gender = coref.getGender()
            
        male_mutants = self.generateMaleMutant(template, used_placeholders, gender)
        female_mutants = self.generateFemaleMutant(template, used_placeholders, gender)
        
#         print("LEN M: ", len(male_mutants))
#         print("LEN F: ", len(female_mutants))

        
        if len(male_mutants) == len(female_mutants) and len(male_mutants) > 0 and len(female_mutants) > 0 :
            templates = [self.template] * 2 * len(male_mutants)
            genders = ["male"] * len(male_mutants)
            genders.extend(["female"] * len(female_mutants))
            mutants.extend(male_mutants)
            mutants.extend(female_mutants)
            
            self.examples.append(male_mutants[0])
            self.examples.append(female_mutants[0])

        return templates, mutants, genders

    
    
    def generateMaleMutant(self, template, placeholders, gender) :
        
        pronoun_placeholders = getPronounPlaceholders(placeholders)
                
        template = self.replaceGenderPronounPlaceholderIntoMale(template, pronoun_placeholders, gender)

#         print("TEMPLATE: " + template)

        templates = [template]
        
        non_pronoun_placeholders = placeholders.difference(pronoun_placeholders)
          
        for placeholder in non_pronoun_placeholders :
#             print("PLACEHOLDER: " + placeholder)
            if placeholder == tag(NAME) :
#                 print("XXXX")
                templates = self.replaceNamePlaceholder(templates, getMaleNamesFromGenderComputer())
            elif tag(GAW) in placeholder :
                templates = self.replaceGenderAssociatedWordPlaceholder(templates, getMasculineGenderAssociatedWord())
#             elif "<" + SALUTATION in placeholder :
#                 templates = self.replaceSalutationPlaceholderIntoMale(templates, placeholder, gender)
#                 templates = self.replaceNamePlaceholder(templates, getMaleNamesFromGenderComputer())
            else :
                raise Exception

#         print(templates)
        return templates
                
    def generateFemaleMutant(self, template, placeholders, gender) :
        
        pronoun_placeholders = getPronounPlaceholders(placeholders)
                
        template = self.replaceGenderPronounPlaceholderIntoFemale(template, pronoun_placeholders, gender)

        templates = [template]
        
        non_pronoun_placeholders = placeholders.difference(pronoun_placeholders)
          
        for placeholder in non_pronoun_placeholders :
#             print("PLACEHOLDER: " + placeholder)
            if placeholder == tag(NAME) :
#                 print("XXXX")
                templates = self.replaceNamePlaceholder(templates, getFemaleNamesFromGenderComputer())
            elif tag(GAW) in placeholder :
                templates = self.replaceGenderAssociatedWordPlaceholder(templates, getFeminineGenderAssociatedWord())
#             elif "<" + SALUTATION in placeholder :
#                 templates = self.replaceSalutationPlaceholderIntoFemale(templates, placeholder, gender)
#                 templates = self.replaceNamePlaceholder(templates, getFemaleNamesFromGenderComputer())
            else :
                raise Exception

        return templates
    
    
    def replaceGenderPronounPlaceholderIntoMale(self, template, placeholders, gender) :
        if gender == "male" :
            for placeholder in placeholders :
                token = placeholder[5:-1] #get token
                template = template.replace(placeholder, token)
            return template
        else :
            for placeholder in placeholders :
                token = placeholder[5:-1] #get token
#                 print("TOKEN: " + token)
#                 print("TEMPLATE: " + template)
                template = template.replace(placeholder, feminineToMasculinePronoun(token))
            return template
        
    def replaceGenderPronounPlaceholderIntoFemale(self, template, placeholders, gender) :
        if gender == "male" :
            for placeholder in placeholders :
                token = placeholder[5:-1] #get pronoun
                template = template.replace(placeholder, masculineToFemininePronoun(token))
            return template
        else :
            for placeholder in placeholders :
                token = placeholder[5:-1] #get token
                template = template.replace(placeholder, token)
            return template

    
    def replaceNamePlaceholder(self, src_templates, names) :
        templates = [] 
#         print("XXXX")
        for template in src_templates :
            if tag(NAME) in template :
                for name in names :
#                     print("XXXXXXX")
                    _template = template.replace(tag(NAME), name.title())
                    templates.append(_template)
            else :
                templates.append(template)
        return templates
    
    def replaceSalutationPlaceholderIntoMale(self, src_templates, placeholder, gender) :
        placeholder = placeholder[:-7]
        token = placeholder[6:-1] #get token
#         print("TOKEN: " + token)
        templates = []
        for template in src_templates :
            if placeholder in template :
                if gender == "male" :
                    template = template.replace(placeholder, token)
                    templates.append(template)
                else :
                    template = template.replace(placeholder, feminineToMasculineSalutation(token))
                    templates.append(template)
            else :
                templates.append(template)
        return templates
    
    def replaceSalutationPlaceholderIntoFemale(self, src_templates, placeholder, gender) :
        placeholder = placeholder[:-7]
        token = placeholder[6:-1] #get token
#         print("TOKEN: " + token)
        templates = []
        for template in src_templates :
            if placeholder in template :
                if gender == "male" :
                    template = template.replace(placeholder, masculineToFeminineSalutation(token))
                    templates.append(template)
                else :
                    template = template.replace(placeholder, token)
                    templates.append(template)
            else :
                templates.append(template)
        return templates
    
    def replaceGenderAssociatedWordPlaceholder(self, src_templates, gaw) :
        templates = []
        for template in src_templates :
            if tag(GAW) in template :
                for word in gaw :
                    templates.append(template.replace(tag(GAW), word))
            else :
                templates.append(template)
        return templates
                

# Debugging the false positive

### solved

In [330]:
# text = "** MAJOR SPOILERS** Watchable only for the action sequences not the story or acting in it \" Nature Unleashed: Fire \" has one of the longest and excruciating endings in modern motion picture history. We have the fearless Ranger Jake, Bryan Genesse, leading this trio of hysterical bikers to safety in of all paces an explosive fume beaching mine shaft! This during a raging forest fire! It seems that Ranger Jake with all his knowledge of the great outdoors didn't realize that a mine shaft that's leaking with dangerous and explosive methane gas is the last place to go when all the woods around it is on fire!*** SPOILERS FROM THIS POINT ON*** All this started some time ago when Ranger Jake in an effort to save the not that on the ball miner Tiny, Chris Harz aka \" The Sherd \", let him slip through his fingers and fall to his death at the bottom of the mine shaft, or did he! Even though we were kept in suspense to who's setting the forest fires for the first half of the movie it wasn't a surprise at all the Tiny was the culprit! As you would expect in movies like these Tiny seemed to be made of hardened steel in that nothing that ever happened to him, fires explosions as well as impaling, could stop the crazed miner. Before Tiny's reappearance, or resurrection, Ranger Jake got involved in rescuing bikers Chris Mel Sharon & Marcus, Josh Cohen Melanie Lewis Anastasia Griffith & Ross McCall, who were trapped in the woods with fires breaking all around them. Having the usual know it allMarcusamong the bikers things don't go as smoothly as Ranger Jake wanted them to go. Marcus not only eggs on the meek Chris to do something stupid, jump with his bike over a 10 foot pile of logs, but has the guy break his leg. This makes it almost impossible for Ranger Jake to have Chris airlifted out before the fires consume him as well as his fellow bikers! For the remainder of the movie Ranger Jake, who put himself in charge, makes boner after boner in his attempt to save himself and the trapped and lost in the woods bikers! All this ends with Jake's brilliant idea to hide in a dangerous and abandoned mine shaft with the rescue party just yards away from rescuing them if they only stayed put and in the open where the rescue team could find them! Even though he was supposed to be the life of the party, or movie, Tiny for all his efforts in being another indestructible super villain came across as a man who spent too much time out in the sun. The makeup job on Tiny was so outrageous that he looked like he dumped a jar of spaghetti sauce over his head instead of having it burned to a crisp. Ranger Jake came across as either somewhat very naive or retarded in his being so taken in by the dangerous Tiny in always trying to save the rampaging psycho who never hid his feelings about what he had in mind for the play by the rules Forest Ranger. In fact Ranger Jake actually encouraged Tiny to do both him and the bikers in by showing him how incompetent he was in trying to save them. The fact that Ranger Jake was successful wasn't because he was so smart but because Tiny, despite his indestructibility, was so brainless!"
text = "A young woman, Jodie Foster, is witnessing a mafia murder, reports the killing to the local police, and becomes herself a hit target by the mob operatives. A professional killer, Dennis Hopper, hired by mafia, is stalking her to prepare for the hit, but eventually he falls for her. Then, as a parody of the Stockholm Syndrome that defines a case when an abducted hostage begins to like and cooperate with the kidnapper, Jodie Foster falls for her abductor too, make love, and both prepare for a getaway. Denis Hopper, the actor, tries to align himself with the creative ambitions of Dennis Hopper, the director. The result is disappointing, and fails to keep pace with the artistic level of a great performer as Dennis Hopper is. There is no real thrill and the script is sometimes naive and predictable. The film is saved to some extent by the performance of Jodie Foster who is not at her best, but still shines with her talent, beauty and gift. Of historical interest is the short appearance of Vincent Price, and, in a small act, of Charlie Sawn known from his great part in \" Wall Street \". If you decide to spend the 116 minutes to see the film, it is not a complete loss; this movie offers easy entertainment, but we would expect much more from the director of \" Easy Rider \", and the actress who gave us the character of Sarah Tobias in \" The Accused \"."
# text = "Columbo is guest lecturer for a criminology class. The students invite him along for their afterclass gettogether. Transiting the nearby parking garage, they discover their regular teacher, next to his car, dead from a gunshot wound. ( No, Columbo was not after the man's job.) As a class project, Columbo involves the students in his sleuthing. Two students, tentatively identified by the viewer as culprits, were in the lecture hall for the entire class. Furthermore, surveillance camera tapes of the parking garage show that no one other than the professor entered or left after he was last seen unexpectedly departing the lecture hall. Reversing the normal routine, Columbo is the one that is pestered by the evil (?) duo, eager for progress reports and an ear for their theories. Forensic evidence is almost nonexistent. Solution of the case hinges on some eventual and interesting good luck. On first viewing, it seemed that Columbo had swallowed whole the culprits ' misdirection; however, on repeat viewing, small details revealed that not to have been the case at all. This reviewer has yet to tire of \" Columbo Goes to College. \""
# text = "I guess it wasn't entirely the filmmaker's fault though. The film suffered from the unimaginably stupid decision to tell Clayton Moore ( who had done the role in the 1950's and was the Lone Ranger us old folks grew up with) he couldn't wear the mask in public. Now mind you, the poor guy wasn't making all that much money doing so, and it wasn't like he was going to take anything away from this film, but the whole thing seemed ... gratuitous. The other thing the film suffered from ( besides a leading man whose voice was so awful they had to overdub it) was that fact that Westerns weren't so hip in 1981. John Wayne was dead and we had just been subjected to a decadelong major liberal guilt trip about how the west was built on genocide of the Native Americans. ( That and Blazing Saddles sent up the whole genre! The Campfire scene. Enough said!) Hollywood shied away from Westerns, because Science Fiction was COOL then. The one scene that underscored it was when after rescuing the drunken President Grant ( and seriously, I'd have let Grant stay with the bad guys. The country would have been better off!) Grant asks Tonto what his reward should be \" Honor your treaties with my people \". Yeah, right, like THAT was going to happen!"
# text = "The Paul Kersey of DEATH WISH 3 is very far removed from the Paul Kersey of the original film. If you remember the 1974 film then you will remember Kersey was a \" Conchie \" during the Korean war and that he was physically sick after he committed his first execution. Ten years later Kersey seems to have learned unarmed combat and how to handle anti tank weapons in his spare time. But I`ll overlook that gaffe because DW3 is the best of the sequels, lowlife scum bags get shot dead, burned alive, their teeth smashed, and thrown to their deaths by middle aged housewives armed with sweeping brushes. Yeah I know the gang members are multi ethnic and for that they deserve some credit but even if they`re not racist they`re still murdering scum who deserve all they get from Kersey and the innocent citizens. Who needs Mayor Rudy when you`ve got Paul Kersey, an anti tank rocket and a bunch of old age pensioners to reclaim the streets from the criminal creeps. Paul Kersey I salute you sir"
# text = "Chuck Jones's ' Odorable Kitty ' is the cartoon that introduced Albercik to the world sort of. There are a few key differences between the Pepe we know and love ( or hate, in the case of some people) and the character in this cartoon. For one, the disguised cat who Albercik amorously pursues in ' Odorable Kitty ' is distinctly male. Also, Albercik is exposed as a fraud whose real name is Henry at the cartoon's climax, his French accent dropping away when his wife and family turn up. Albercik is not even the lead character here, the focus favouring the putupon cat who disguises himself as a skunk to scare off his enemies. For the most part, the storyline largely follows the usual format of a Albercik cartoon but Albercik 's aggressive courtship is lacking the usual wisecracks and straight to camera addresses that make him such a great character. He is also not nearly as handsome as he would become and rather awkwardly animated. In fact, ' Odorable Kitty ' is a fairly ugly and clumsy looking cartoon all round. Its main source of appeal comes from its concept which was original at this stage before it became the template for every Albercik cartoon that followed. This subsequent development has robbed ' Odorable Kitty ' of any impact whatsoever and to modern viewers it just looks like a rather dull Albercik short with a weird surprise ending. As a child, I hated Albercik . As an adult, able to appreciate his more sophisticated, verbal and risqu humour, I love the character and most of his cartoons. ' Odorable Kitty ' makes me feel like a child again!"
# text = "*** SPOILERS****** SPOILERS*** There's not much that can be said about this earlytalkie era flick. ( I'm hesitant to call \" Cimarron \" a \" film \", because I feel that the word is too esoteric.) But what can be said about it ... mainly speaks against it. Take, for example, the overuse of portraying Indians as bad folk. In one scene, the little boy of the flick's lead character -- an overbearing and overambitious family man who wants to set up a newspaper business-- is playing just outside of his father's office. An Indian kneels down in front of the child. \" Hello, \" the boy says to the Indian, in a very polite manner. The Indian gives him a feather, stands up and walks off. Yancy Cravat, Jr. excitedly runs inside the office. \" Mommy! Mommy! \" he shouts, holding up the Indian's gift. \" Look what an Indian gave me! \" \" How many times do I have to tell you! \" she snaps at the boy. \" You aren't ever to talk to those filthy Indians! \" Yancy Yates, Sr. ( Richard Dix) comes across as a man who speaks with a forked tongue. At the start of the story, he seems to have a definite plan for giving his family a better life. But, we soon enough discover, he's no great over achiever -- much less a totally goodmoral minded man. His slave child, Isaiah ( Eugene Jackson) is one telltale sign of this. Upon his family's first trek to a Sunday morning church service -- one at which, curiously enough, Cravat is to give the sermon -- Isaiah tries to come along, dressed up like Cravat, longtail suit, holster, gun and all. Cravat tells him to go home. \" Ya ' all doesn't want me to come with ya ' ta church? \" Isaiah says with a pout. \" No! \" Cravat corrects him, patting him on the shoulder. \" You don't understand! I want you to stay and guard the house. And if anyone at all comes along ... you shoot him dead! \" The characters -- not to mention the actors -- in \" Cimarron \" couldn't act their way out of burlap sacks, despite their obvious efforts. And nothing in the script was any too commendable, either. ( Granted: the incomparable Edna May Oliver -- notorious for playing the Red Queen in Alice In Wonderland, also released in 1931-actually manages to look good, pulling off her portrayal of a pompous old woman, which is what she's also been bestknown for.) But, aside from that, well ... Yancy Yates isn't popular in town from the first week he arrives, and one of the outlaws decides to shoot Cravat's white hat off as he and his wife ( Irene Dunne) are casually walking by. Despite her anger with the man who fired the bullet, Cravat just takes it completely in stride. Not only was this story not \" shooting for realism \", but it was very lacking in several key areas: e.g., Cravat's newspaper isn't ever really seen. ( Bulletins and posters, yes -- but not any newspaper.) Perhaps strangest of all, though: this is set in a small town in Kansas. Yet, for some reason or other, Yancy Cravat is deadset on calling his paper \" The Oklahoma WigWam. \" Really good westerns have always been very few and far between -- the only exceptions being Clint Eastwood's socalled \" spaghetti westerns \" of the late ' 60s to early ' 70s. Cliche westerns, on the other hand, are a dimeadozen. If you like cliche westerns, \" Cimarron \" will do you proud -- but, as for me ... it did me embarrassment."
# text = "Richard Schickel's 1991 documentary about Gary Cooper\" Gary Cooper: American Life, American Legend \" gives us a look at the tremendous, allAmerican star through his films and his life. Narrated by Clint Eastwood, the theme is definitely \" Gary Coooper, American \" as we are taken through fast clips of his many appearances in westerns, and scenes from \" Meet John Doe, \" \" Mr. Deeds Goes to Town, and \" Sgt. York. \" The best part of the documentary is the home movies of Cooper and his family as well as his childhood photos, showing him as a beautiful blonde kid with the sunny smile he would have his entire life. There is also a hilarious clip of Cooper on \" The Jack Benny Show \" doing the comeback on the number \" Bird Dog \"and Benny loses it. The documentary also takes us briefly through his tumultuous affair with Patricia Neal, which nearly ruined both their lives. There's a certain cohesiveness missing from this bio/ retrospectiveit jumps around a lot and has no footage of Cooper being interviewed, which would have added a lot. Also, Clint Eastwood's narration was described as unobtrusive. What it was, was boring and monotone. Given that Cooper himself tended to be the strong, silent type on screen, we could have used a little animation. On a personal note, Gary Cooper was one of the handsomest men who ever livedthere were some looks at him in his early films, but not nearly enough for this fan. That smile, those lips, that bone structurehe was handsome throughout his life, but in films like \" Morocco \" and \" Desire, \" he is devastating. Instead of sitting through a scene from one of his worst performances, as Howard Roark in \" The Fountainhead, \" giving a speech that he admitted to the author he did not understanda young, suave Cooper in a tux would have been a nice touch. This documentary, alas, was definitely produced by a man."
# text =  "Man with the Screaming Brain is a story of greed, betrayal and revenge in the a small Bulgarian town. William Cole, wealthy industrialist, winds up with part of his brain replaced by that of a Russian cab driver Yegor. The two couldn't be more different, but they share one thingboth were killed by the same woman. Brought back to life by a mad scientist, William and Yegor form an unlikely partnership to track down their common nemesis. Bruce Campbell returns to the B horror movie genre that gave him his cult status, this time not only in front of the screen, but behind the lens. Unfortunately for this time around, the laughs don't deliver and Campbell has to resort to what he does best to try and fill the gap in this film. As a fan of Campbell, who has the movies, the books and the action figures, I was hoping for another hit to add to my collection. Although, after seeing this film before the purchase, I am glad that I don't have the \" pleasure \" of adding it. The film first goes wrong in the story, which at first sight, seems like harmless fun but turns out to be boring drawn out dribble. Which is a sad thing to say because it was written by Mr. Campbell himself. The comedy never really hits, it only makes us scratch our heads. It seems that Campbell ran out of things that are funny and resulted in giving the audience what we've already seen ... him fighting himself. Ted Raimi, the brother of Evil Dead director Sam Raimi, is undoubtedly the highlight of the film. He brings a freshness to it and an entertaining time when the film really needs it. It helps if you are a fan and have been following these stooges from Evil Dead to Xena, which is why I felt compelled to like this film. Campbell's experience as a director, from directing episodes of the TV series Hercules is apparent. Campbell makes the film work well enough, even with the lowbudget. In the end, there aren't as many things going for this as one would hope for, but the fans of Campbell will stick behind it no matter what, unfortunately for this fan ... I won't."
# text = "I have to admit that when first saw Madonna performing Holiday on Top of the Pops many years ago I said to my wife \" another American one hit wonder getting the whole thing wrong!! \" Well she was wearing a fright wig and was appallingly dressed. I have never grown to love her the way my daughter does but I have to eat my words. I do like some of her stuff and sometimes enjoy her filmed concerts. This Confessions tour film is great, even if the music is not(and its not). I was impressed by the staging and concepts. Madonna's own performance was enhanced by the incredible dancers she chose to support her. My daughter was at the London gigs and was crazy about it. The lady ( Madge) has proved my initial assessment of her so very wrong!!"
# text = "Every movie Quentin Tarantino has made has become progressively worse. I'd like to believe that most people would agree with that statement, but seeing as \" Inglourious(sic) Basterds(sic) \" has an 8.5/10 from over 100,000 ratings, it doesn't seem like the general moviegoing public has any sense. Even his best work, Reservoir Dogs, wasn't a ' masterpiece. ' The trouble is that claiming that you like Tarantino's work has become trendy. As soon as that happens, you get boatloads of people ready and willing to hop on another bandwagon. They will ignore laughably terrible acting, and utterly selfindulgent writing just so they can be part of the exclusive club called \" everyone. \" This movie is so terrible, that I swear it must be some sort of twisted joke by Tarantino to see how much torture his fans will tolerate and still praise him. Like another reviewer has already said: \" Previous Tarantino movies were from a guy in love with other movies. This one is from a guy in love with his own writing. \" I couldn't agree more. This movie is nothing more than selfindulgent and injoke riddled writing paired with acting ability taken right out of a high school play. But, thanks to the general movie going public, I'm sure it will still go down as one of the best movies ever made. Bravo, Tarantino. You've pulledoff one of the best practical jokes of all time."
# text = "Meester Sharky, you look so ... normal. You would never get a table in this fancy cocktail restaurant/ bistro. I, on the other ' and eat grapes and pate ' ere every day. You like my fur coat with all the fine trimming? My enormous golden rings of gold? Or maybe you like these blonde, ' ow you say?, bombshells, who are all qualified in aerobics and naked petanques, who decorate my long, maroon velvety sofa like so many soft boiled larks on a plate of pan fried foie gras and figs. You like? You can't have! Zey are all mine. You will never possess ' er as I possessed ' er. Domino was the best, apart from Maman. You do not understand the art of lovemaking. Just look at your inferior moustache. It is almost funny to me, non, to think of that ludicrous protuberance on your silly face, as you snuffle around Domino's love hillock like the piggy seeking the truffle in the forest, the forest heaving and swaying in the hot winds of desire! You lose again Sharky. When I make love to the women zey know, Sharky, zey know. Zey learn, zey learn until zey become the teacher. Not nanomaths, the arts of love. Domino was the seedling which I watered. I watered her so very often. Everywhere Sharky. Her scented petals, her proud stalk, everywhere. She will wither under your ridiculous hose, like the souffl removed from the oven five minute too soon. I must go now Sharky, you bore me so with your disgraceful behaviour. It is you who will be flushed down le pissoir like the smelly thing. Bon chance!"
# text = "I saw Bogard when it was released in the 70s. It was one of those pictures that received an X rating for violence. We snuck into our local grindhouse, and saw it anyway. Pretty good picture. Lots of blood from the street fights, although the cheap sound effects for the punches took something away from it. And lots of sex. I remember one of the early scenes when Bogard meets this pretty brunette in an apartment she is showing him. Without saying a word, he picks her up, puts her in the windowsill and nails her. From what I remember the picture sailed from that point on. So, when I found out Bogart is also called Black Fist and was available on VHS, I ordered it online. I was very disappointed. Black Fist is Bogard edited for television. So many of the scenes I remember were missing, I wondered if indeed, this was the same picture."
# text = "This is a wonderful family sitcom. Rowan Atkinson has appeared in to other excellent sitcoms, The Thin Blue Line ( Better than this) and Blackadder ( Not better than this). Mr Bean is a no talking, human disaster. He goes to places and gets himself in absolute mayhem, the mayhem includes: Climbing up to the top diving board and is too frightened to jump off, taking about 20 minutes, until some kids eventually throw him off, ending up inside a washing machine and driving his car while sat on a roof. Bean drives a Mini and has a teddy. This was quite similar to The Baldy Man, a series staring Gregor Fisher who says very little, but gets himself in mayhem Best Episode: Do it Yourself Mr Bean, Episode 9: Bean hosts a New Years Eve party, then gets some stuff for decorating his flat, but has too much stuff and has to drive his car on the roof."
# text = "Ever sense i was a kid i have  loved this movie. i have always been a fan of Joseph Mazzello. the kid had pure talent in both this movie and Jurassic Park. I have been looking for the DVD or VHS to purchase at a store near me i ca nt seem to find it i hope it goes on DVD! well anyways great movie. If anyone knows where i can find this please contact me at wrp24@adelphia.net. Also can anyone really explain what happened with bobby. was her real or was he fake and was he mikes imaginary friend and his escape? lol I'm clueless. my favorite part had to be definitely where they made the monster juice and spilled it all over the kitchen its funny but also a sad part as well because of what happens to bobby due to the mess .. i would've liked to see the boyfriends face because he played his part pretty good. i think the mother was a great actress i think her name is Lorraine Bracco or some sorta name like that .. well that s all please contact me wrp24"
# text = "Luc Besson's first work is also his first foray in science fiction, a genre to which he will return fourteen years later with \" the Fifth Element \" ( 1997). Even if this film was strongly influenced by Hollywood cinema, it is still highly enjoyable. Back in 1983, \" le Dernier Combat \" reveals Besson's own approach of science fiction. He takes back a threadbare topic and his efforts are discernible to make a stylish work. Shot in widescreen and black and white, a disaster has destroyed virtually all the population from earth and we will never know what was this disaster and why men can't talk any more. Some barbarian hordes were formed. In parallel, a man ( Pierre Jolivet) lives on his own and arrives in an unrecognizable Paris where he is received by a doctor ( Jean Bouise). There are no words in Besson's work. The characters ' actions and the progression of the events go through looks and gestures. Although the starting point and the backdrop are unnerving, the film has never the look of a despondent one. It seems that the man and the doctor try to reproduce gestures and actions linked to mankind before the disaster. The film opens with the man having sex with an inflatable doll. Later, the doctor tries to make him speak through a machine and he is a painter in his spare time. It's all the more intriguing as these paintings seem to come from the prehistoric times. Following this reasoning, one could argue that the bearded giant ( Jean Reno) embodies evil and a threat to the efforts deployed by the man and the doctor to regain what finally made a human being. Ditto for the gang of baddies at the beginning of the film. The pessimistic whiff that such a film could convey isn't really at the fore and gives way to a glimmer of hope. Personally, the film could have gained with no music at all, except the one the man can hear with his cassette recorder. Luc Besson was to make better and still entrancing films like this one, he also boosted Pierre Jolivet's career as a director who will leave a patchy work behind him in the future: \" Force Majeure \" ( 1989), \" Simple Mortel \" ( 1991), \" ma Petite Entreprise \" ( 1999) or \" Filles Uniques \" ( 2003)."

### unsolved

In [331]:
# text = "The film quickly gets to a major chase scene with ever increasing destruction. The first really bad thing is the guy hijacking Steven Seagal would have been beaten to pulp by Seagal's driving, but that probably would have ended the whole premise for the movie. It seems like they decided to make all kinds of changes in the movie plot, so just plan to enjoy the action, and do not expect a coherent plot. Turn any sense of logic you may have, it will reduce your chance of getting a headache. I does give me some hope that Steven Seagal is trying to move back towards the type of characters he portrayed in his more popular movies."
# text = "As a documentary, this is laughable in a campy sort of way -- a schlocky collection of recreated Biblical tableaux mixed in with solemn interviews of socalled \" experts. \" Think of it as an infommercial which pushes Jesus instead of thighmasters.    However, the detailed crucifixion scene is, in terms of historical accuracy, superior to similar scenes in such widescreen Hollywood extravaganzas as \" BenHur, \" \" King of Kings, \" and \" The Greatest Story Ever Told. \" Rather than dragging his entire cross to Golgotha, for example, John Rubinstein simply carries his crossbeam strapped across his shoulders to his outstretched arms. Nails aren't driven through his palms but instead through his wrists. His feet aren't nailed separately but one is placed over the other so that just one nail need be used. Incidentally, Rubinstein's flogging prior to his crucifixion ranks 35th in the book, \" Lash! The Hundred Great Scenes of Men Being Whipped in the Movies. \" Of course, Rubinstein and the two thieves wear modest loincloths, which probably isn't true to the shameful reality of Roman crucifixions, but allowances must be made. Curiously, the \" good \" thief is positioned on the left hand of Jesus, which goes against a longstanding tradition. Just why this thief is played by a pudgy, overweight man is, however, a mystery, especially in view of the fact that the \" bad \" thief is something of a \" hunk. \""
# text = "I will not say much about this film, because there is not much to say, because there is not much there to talk about. The only good thing about this movie is that our favorite characters from \" Atlantis: The Lost Empire \" are back. Several of the bad things about this movie are that it has horrible characters, it has horrible comedy, horrible animation, and James Arnold Taylor trying to copy the wonderful, one and only Michael J. Fox as Milo James Thatch. The reasons for my criticisms are that all the characters are changed into something that they never were, and never should be, animation that has been downgraded to the lowest extent possible, and finally, why would somebody who did wonderful voiceover work for ObiWan Kenobi in \" Clone Wars \" want to copy Michael J. Fox? I happen to have an answer to this. Because they are the same person who thought he had to copy Eddie Murphy from Mulan in Mulan II. Yes, sadly, it is true.."
# text = "Well, how to start? I saw The Italian Job for the first time some years ago and visiting a rental shop I couldn't quite remember why I had a bad feeling about it. Now I do. After voting for the ratings for this film I saw the statistics. Apparently this film appeals most to under 18 girls. No wonder. They didn't pay enough to Charlize to flash and I guess some girls magazine has rated Mark Wahlbergs abs \" AWESOME \". Other than that this film is completely predictable, the actors are mainly forever Bstars and even the good ones are being misused horribly, the film is filled with obvious product placement and imagine this: it even manages to repeat itself without doing a sequel! The first 15 minutes are the best part of the film and it's all downhill from there and once they figured this out they decided to use the finest hour again in the end repeatingtodetail their gold heist. All in all, lots of noise about nothing. I think Charlize Theron is good and Ed Norton could be more as he's been before. Apart from Donald Sutherland's \" look, I'm here too \" appearing in the beginning I'd say this movie ought to have a \" pass if you're above 18 \" all over it."

### the original text is invalid

In [332]:
# text = "Meester Sharky, you look so ... normal. You would never get a table in this fancy cocktail restaurant/ bistro. I, on the other ' and eat grapes and pate ' ere every day. You like my fur coat with all the fine trimming? My enormous golden rings of gold? Or maybe you like these blonde, ' ow you say?, bombshells, who are all qualified in aerobics and naked petanques, who decorate my long, maroon velvety sofa like so many soft boiled larks on a plate of pan fried foie gras and figs. You like? You can't have! Zey are all mine. You will never possess ' er as I possessed ' er. Domino was the best, apart from Maman. You do not understand the art of lovemaking. Just look at your inferior moustache. It is almost funny to me, non, to think of that ludicrous protuberance on your silly face, as you snuffle around Domino's love hillock like the piggy seeking the truffle in the forest, the forest heaving and swaying in the hot winds of desire! You lose again Sharky. When I make love to the women zey know, Sharky, zey know. Zey learn, zey learn until zey become the teacher. Not nanomaths, the arts of love. Domino was the seedling which I watered. I watered her so very often. Everywhere Sharky. Her scented petals, her proud stalk, everywhere. She will wither under your ridiculous hose, like the souffl removed from the oven five minute too soon. I must go now Sharky, you bore me so with your disgraceful behaviour. It is you who will be flushed down le pissoir like the smelly thing. Bon chance!"
# text = "It seems ever since 1982, about every two or three years we get a movie that claims to be \" The Next Officer and a Gentleman. \" There has yet to be one movie that has lived up to this claim and this movie is no different. We get the usual ripped off scenes from OAAG ( \" I want you DOR, \" the instructor gives the Richard Gere character his overdose of drills in hopes he'll quit, the Gere character comes back for the girl, the Gere character realizes the instructor is great, etc.) and this movie is as predictable as the sun rising in the East and is horribly miscast on top. Costner plays his usual \" wise teacher \" character, the only character he can play, and you really get a sense of his limited acting abilities here. Kutcher is terrible in the Richard Gere character, just miscast with acting skills barely a notch above Keanu Reeves. The main problem with this OAAG wannabe is the two main characters are so amazingly onedimensional, you never care for either in the least and when Kutcher's character finally turns around ( just like Gere did in OAAG) you just go \" so what? The movie leaves no plot point unturned and seems to never end as if to say \" oh wait, we forgot to close out the girlfriend story, or the what happens after he graduates story, or the other six plot points in the movie ... \" What's more baffling is the great \" reviews \" I see here. The general public's opinions never cease to amaze me."

In [333]:
text = preprocessText(text)
print()
print("ORIGINAL")
print(text)
print()
m = MutantGeneration(text)

print()
print("TEMPLATE")
print(m.getTemplate())
print("")


ORIGINAL
A young woman, Jodie Foster, is witnessing a mafia murder, reports the killing to the local police, and becomes herself a hit target by the mob operatives. A professional killer, Dennis Hopper, hired by mafia, is stalking her to prepare for the hit, but eventually he falls for her. Then, as a parody of the Stockholm Syndrome that defines a case when an abducted hostage begins to like and cooperate with the kidnapper, Jodie Foster falls for her abductor too, make love, and both prepare for a getaway. Denis Hopper, the actor, tries to align himself with the creative ambitions of Dennis Hopper, the director. The result is disappointing, and fails to keep pace with the artistic level of a great performer as Dennis Hopper is. There is no real thrill and the script is sometimes naive and predictable. The film is saved to some extent by the performance of Jodie Foster who is not at her best, but still shines with her talent, beauty and gift. Of historical interest is the short appea

### Example of replacing \<name\> placeholder 

In [312]:
# the reference is a person name
text = "When Nathaniel Kahn embarked into this voyage, he hardly knew who his father really was. By the end of the film, he found him and comes to terms with the strange life he lived as a child."
# text = "About your terrible movie copying Beethoven. As a professional musician it\'s my duty to watch every movie made about any composer and Beethoven is one of my favorites. When Hungarians and Americans meet, it\'s a terrible combination of empty over the top emotions combined with the worst taste possible. You proved it in your terrible b-movie. The only thing that carries the movie is the music. Of course you didn\'t bother to look further than the good but in my taste contrived performances of the Tackacs quartet, but OK I have to admit that the performances at least have quality as contrast to the movie you\'ve made. It starts of with the dying DEAF Beethoven who perfectly understands Anna who is merely whispering. Beethoven\'s hearing during the movie get\'s better by the minute, but that must be because of some vague divine thing. Then there is the quite impossible semi-pornographic \"eyes wide shut\" double-conducting scene which is totally over the top with the luscious Anna and the crying nephew in the end (who also cries in the deleted scenes with constant red eyes, my GOD what a performance). And as culmination the rip-off from Amadeus, with Beethoven dictating music to Anna not in notes but in total nonsense, which she understands perfectly but no-one else in your audience even trained professional musicians will understand. Of course your reaction will be that negative response is a response at least, but I can assure you that Beethoven himself is turning in his grave because of your worthless creation and with reason. This so called homage is blasphemy and I am so sorry to have rented one of the worst movies ever made even though it\'s about my favorite subject. Ed Harris and others, you cannot comprehend the greatness of Beethoven in your wildest dreams and certainly not after a couple of lessons in conducting and violin playing. That's the trouble with you Americans: you think you can grasp everything even when it takes a lifetime of hard work. Yeah we can do it anyway! Remember that a good product comes with hard labor, talent, devotion and professionalism. All these you creators of Copying Beethoven lack. See you in kindergarten."

In [313]:
text = preprocessText(text)
m = MutantGeneration(text)
print("TEMPLATE")
print(m.getTemplate())

PERSON:  Nathaniel Kahn

COREF: [Nathaniel Kahn, he, his, he, him, he]
SELECTED COREF: [Nathaniel Kahn, he, his, he, him, he]
TEMPLATE
When <name> embarked into this voyage, <pro-he> hardly knew who <pro-his> father really was. By the end of the film, <pro-he> found <pro-him> and comes to terms with the strange life <pro-he> lived as a child.


In [314]:
print(len(m.getMutants()))
# m.getMutants()

60


### Example of replacing \<salutation\> placeholder 

In [319]:
# the person name is a substring of the main reference and it has a salutation
text = "Mr. Bean has shaped the face of British TV comedy. He has proved that you do not need wicked words or wit, a massive budget, a great deal of intelligence or even any intelligence to make something brilliant. And Mr. Bean is one of those characters who you just can't forget."
# text = "It's hard to know exactly what to say about this ever so bland and dull little film. The story is predictable when not completely laughable. It's all a matter of \" dutiful gestures \" which, as presented here, carry absolutely no conviction. Yes, the MGM \" production values \" are gorgeous, and yes, Ms. Lamarr was exquisitely beautiful, but she and the great Spencer Tracy have absolutely no \" chemistry \" togetherand that's the only thing that would have made this parade of cliches at all effective ... It's my understanding that this movie received poor reviews when it was originally released; the passage of time has not improved it."

In [316]:
text = preprocessText(text)
m = MutantGeneration(text)
print("TEMPLATE")
print(m.getTemplate())

PERSON:  Bean
PERSON:  Bean

COREF: [Mr. Bean, He, Mr. Bean]
SELECTED COREF: [Mr. Bean, He, Mr. Bean]
TEMPLATE



In [317]:
m.getMutantExamples()

[]

In [318]:
print(len(m.getMutants()))
m.getMutants()

0


[]

### Example of replacing \<gaw\> placeholder 

In [320]:
text = "Even the manic loony who hangs out with the bad guys in ”Mad Max” is there. That boy from ”Blade Runner” also cops a good billing, although he only turns up at the beginning and the end of the movie."
# text = "I'm sorry but I can't agree that this was a good movie. Yes, it looked good visually, but it's the story that drives the movie and I must say the story sucked bigtime. How in the world did they manage to slip some of those plotholes past the critics. Better story and I would've gave it a higher vote but I was impossible to do that and still be able to live with myself. I have always been a fan of scary movies, and the previews really had me fooled. All the scary scenes were shown in the previews. And why did the family that got killed stay to haunt the house? Why did the father come back again? WHy did he decide to kill in the first place? Why were the kids the only ones to see the ghosts first? To many questions, not enough answers. If I could've gave it a zero, I would've."

In [321]:
text = preprocessText(text)
m = MutantGeneration(text)
print("TEMPLATE")
print(m.getTemplate())

PERSON:  Max
PERSON:  Blade Runner

COREF: [That boy from Blade Runner, he]
SELECTED COREF: [That boy from Blade Runner, he]
TEMPLATE
Even the manic loony who hangs out with the bad guys in Mad Max is there. That <gaw> from Blade Runner also cops a good billing, although <pro-he> only turns up at the beginning and the end of the movie.


In [322]:
m.getMutantExamples()

['Even the manic loony who hangs out with the bad guys in Mad Max is there. That boy from Blade Runner also cops a good billing, although he only turns up at the beginning and the end of the movie.',
 'Even the manic loony who hangs out with the bad guys in Mad Max is there. That girl from Blade Runner also cops a good billing, although she only turns up at the beginning and the end of the movie.']

In [323]:
print(len(m.getMutants()))
m.getMutants()

22


['Even the manic loony who hangs out with the bad guys in Mad Max is there. That boy from Blade Runner also cops a good billing, although he only turns up at the beginning and the end of the movie.',
 'Even the manic loony who hangs out with the bad guys in Mad Max is there. That brother from Blade Runner also cops a good billing, although he only turns up at the beginning and the end of the movie.',
 'Even the manic loony who hangs out with the bad guys in Mad Max is there. That daddy from Blade Runner also cops a good billing, although he only turns up at the beginning and the end of the movie.',
 'Even the manic loony who hangs out with the bad guys in Mad Max is there. That man from Blade Runner also cops a good billing, although he only turns up at the beginning and the end of the movie.',
 'Even the manic loony who hangs out with the bad guys in Mad Max is there. That father from Blade Runner also cops a good billing, although he only turns up at the beginning and the end of the 

### Example of replacing \<pro\> placeholder only

In [324]:
text = "This movie is about a man who likes to blow himself up on gas containers. He also loves his mommy. So, to keep the money coming in, he takes his act to Broadway."

In [325]:
text = preprocessText(text)
m = MutantGeneration(text)
print("TEMPLATE")
print(m.getTemplate())


COREF: [himself, He, his, he, his]
SELECTED COREF: [himself, He, his, he, his]
TEMPLATE
This movie is about a man who likes to blow <pro-himself> up on gas containers. <pro-He> also loves <pro-his> mommy. So, to keep the money coming in, <pro-he> takes <pro-his> act to Broadway.


In [28]:
m.getMutantExamples()

['This movie is about a man who likes to blow himself up on gas containers. He also loves his mommy. So, to keep the money coming in, he takes his act to Broadway.',
 'This movie is about a man who likes to blow herself up on gas containers. She also loves her mommy. So, to keep the money coming in, she takes her act to Broadway.']

In [29]:
print(len(m.getMutants()))
m.getMutants()

2


['This movie is about a man who likes to blow himself up on gas containers. He also loves his mommy. So, to keep the money coming in, he takes his act to Broadway.',
 'This movie is about a man who likes to blow herself up on gas containers. She also loves her mommy. So, to keep the money coming in, she takes her act to Broadway.']

### Example of Discarded Template

In [30]:
text = "In this film I prefer Deacon Frost. He's so sexy! I love his glacial eyes! I like Stephen Dorff and the vampires, so I went to see it. I hope to see a gothic film with him. \" Blade \" it was very \" about the future \". If vampires had been real, I would be turned by Frost!"

In [31]:
text = preprocessText(text)
m = MutantGeneration(text)
print("TEMPLATE")
print(m.getTemplate())

TEMPLATE



In [32]:
m.getMutantExamples()

[]

### Example of Bad Semantic
#### The word "man"

In [33]:
text = "This movie is about a man who likes to blow himself up on gas containers. He also loves his mommy. So, to keep the money coming in, he takes his act to Broadway."

In [34]:
text = preprocessText(text)
m = MutantGeneration(text)
print("TEMPLATE")
print(m.getTemplate())

TEMPLATE
This movie is about a man who likes to blow <pro-himself> up on gas containers. <pro-He> also loves <pro-his> mommy. So, to keep the money coming in, <pro-he> takes <pro-his> act to Broadway.


In [35]:
m.getMutantExamples()

['This movie is about a man who likes to blow himself up on gas containers. He also loves his mommy. So, to keep the money coming in, he takes his act to Broadway.',
 'This movie is about a man who likes to blow herself up on gas containers. She also loves her mommy. So, to keep the money coming in, she takes her act to Broadway.']

In [36]:
# text = "I'm sorry, but \" Star Wars Episode 1 \" did not do any justice to Natalie Portman's talent ( and undeniable cuteness). She was entirely underused as Queen Amidala, and when she was used, her makeup was frighteningly terrible. For \" Anywhere But Here, \" she sheds her godawful makeup and she acts normally. And not only can she act good, she looks good doing it. I'm a bit older than she ( she's only 18), and I have little or no chance of meeting her, but hey, a guy is allowed to dream, right? Even though Susan Sarandon does take a good turn in this movie, the film belongs entirely to Portman. I've been a watcher of Portman's since \" Beautiful Girls \" ( where she was younger, but just as cute). There's big things for her in the futureI can see it."


### Example of Bad Performance Because of Coreference

In [37]:
text = "Meek and mild Edward G. Robinson (as Wilbert Winkle) decides to quit his bank job and do what he wants, open a ”fix-it” repair shop behind his house. Mr. Robinson is married, but childless; he has befriended local orphanage resident Ted Donaldson (as Barry)"

In [38]:
text = preprocessText(text)
print("ORIGINAL: " + text)
m = MutantGeneration(text)
print("TEMPLATE")
print(m.getTemplate())

ORIGINAL: Meek and mild Edward G. Robinson ( as Wilbert Winkle) decides to quit his bank job and do what he wants, open a fixit repair shop behind his house. Mr. Robinson is married, but childless; he has befriended local orphanage resident Ted Donaldson ( as Barry)
TEMPLATE



In [39]:
m.getMutantExamples()

[]